<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Yahoo_Realtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web スクレイピング と感情分析



*   Web スクレイピング: Selenium --> https://ai-inter1.com/python-selenium/
*   感情分析： Transformer --> https://tt-tsukumochi.com/archives/4105



## 1. Selenium による Web スクレイピング

In [ ]:
%%shell
# 更新を実行
sudo apt -y update

# ダウンロードのために必要なパッケージをインストール
sudo apt install -y wget curl unzip libvulkan1 libu2f-udev
# 以下はChromeの依存パッケージ
#wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
#dpkg -i libu2f-udev_1.1.4-1_all.deb

# Chromeのインストール
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

# Chrome Driverのインストール
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
#wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip
wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip
unzip chromedriver-linux64.zip
chmod +x /content/chromedriver-linux64/chromedriver
mv /content/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver

pip install selenium

In [ ]:
%%shell
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
echo $CHROME_DRIVER_VERSION

In [ ]:
%%shell
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
echo $CHROME_DRIVER_VERSION
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip

In [ ]:
!google-chrome --version
!chromedriver --version

In [ ]:
#ライブラリ等のインポート
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
from datetime import datetime, timedelta
import urllib.parse

from bs4 import BeautifulSoup
import time

In [ ]:
# クエリを設定
query = "防衛大学校"
query = urllib.parse.quote(query)
url = f"https://search.yahoo.co.jp/realtime/search?p={query}&gm=w"
url

In [ ]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')

In [ ]:
#インスタンス化
driver = webdriver.Chrome(options=options)
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
# ページの取得
driver.get(url)

# 要素が表示されるまで待機
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@id='graph']/div/div[3]/div[2]/p/span[1]")))
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".recharts-curve.recharts-area-curve")))

In [ ]:
tweets = driver.find_elements(By.CLASS_NAME, "Tweet_body__XtDoj")
print(len(tweets))

## 「もっと見る」＋スクロール

In [ ]:
for i in range(10):
    print(i, '回目の更新')
    # 「もっと見る」ボタンを押す
    driver.find_element(by=By.CLASS_NAME, value="More_text__1eDS4").click()

    #ブラウザのウインドウ高を取得する
    win_height = driver.execute_script("return window.innerHeight")

    #スクロール開始位置の初期値（ページの先頭からスクロールを開始する）
    last_top = 1

    #ページの最下部までスクロールする無限ループ
    while True:
        #スクロール前のページの高さを取得
        last_height = driver.execute_script("return document.body.scrollHeight")

        #スクロール開始位置を設定
        top = last_top

        #ページ最下部まで、徐々にスクロールしていく
        while top < last_height:
            top += int(win_height * 0.8)
            driver.execute_script("window.scrollTo(0, %d)" % top)
            time.sleep(0.5)

        #１秒待って、スクロール後のページの高さを取得する
        time.sleep(1)
        new_last_height = driver.execute_script("return document.body.scrollHeight")

        #スクロール前後でページの高さに変化がなくなったら無限スクロール終了とみなしてループを抜ける
        if last_height == new_last_height:
            break

        #次のループのスクロール開始位置を設定
        last_top = last_height

In [ ]:
tweets = driver.find_elements(By.CLASS_NAME, "Tweet_body__XtDoj")
print(len(tweets))

In [ ]:
tweets[0].text

In [ ]:
for i in tweets:
    print(i.text)

## 2. 感情分析

## WordCloud

In [ ]:
!apt install mecab libmecab-dev mecab-ipadic-utf8
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!apt-get -y install fonts-ipafont-gothic
!pip install mecab-python3 wordcloud japanize-matplotlib

In [ ]:
#import tweepy
import MeCab
import csv
import json
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import japanize_matplotlib
import warnings
import numpy as np
import pandas as pd
import collections

In [ ]:
mecab = MeCab.Tagger()
malist = mecab.parse("すもももももももものうち")
print(malist)

In [ ]:
mecab = MeCab.Tagger()
words = []
for tweet in tweets:
    node = mecab.parseToNode(tweet.text)
    while node:
        word = node.surface
        word_type = node.feature.split(",")[0]

        # "名詞", "動詞", "形容詞", "副詞"の中で選択したものを抽出
        if word_type in ["名詞", "動詞", "形容詞"]:
            words.append(word)
        node = node.next

In [ ]:
#wordcloudで出力するフォントを指定
font_path = '/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf'
txt = " ".join(words)

# ストップワードの設定(意味のない単語が含まれないように設定)
stop_words = ['https', 't', 'co',  'RT']

#解析した単語、ストップワードを設定、背景の色は白
wordcloud = WordCloud(font_path=font_path, background_color="white", stopwords=set(stop_words), width=800, height=600).generate(txt)

fig = plt.figure(figsize=(30, 10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## ネガポジ分析

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

# パイプラインの準備
classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    return_all_scores=True
)

In [ ]:
print(classifier("勇者ヒンメルならそうしました"))
print(classifier("ザクとは違うのだよ、ザクとは！"))
print(classifier("見ろ、人がゴミのようだ！！"))
print(classifier("おまえの物は俺の物、俺の物も俺の物"))

In [ ]:
senti = classifier("おまえの物は俺の物、俺の物も俺の物")
senti

In [ ]:
senti[0][0]['score']

In [ ]:
sentiments = []
for tweet in tweets:
    print(tweet.text)
    senti = classifier(tweet.text)
    print(senti)
    sentiments.append([senti[0][0]['score'], senti[0][1]['score'], senti[0][2]['score']])
print(sentiments)
print(np.mean(np.array(sentiments)))

In [ ]:
df = pd.DataFrame(data = np.array(sentiments), columns=['positive', 'negative', 'neutral'])
df

In [ ]:
df.sum().values

In [ ]:
plt.figure(figsize=(8, 8))
plt.rcParams["font.size"] = 18
plt.pie(df.sum().values, labels=df.columns, counterclock=False, autopct='%0.1f%%', startangle=90)

In [ ]:
# ツイート数の取得
tweet_count = driver.find_element(by=By.XPATH, value="//*[@id='graph']/div/div[3]/div[2]/p/span[1]").text
tweet_count = int(tweet_count.replace(",", ""))

#ツイート推移の取得
time_text = driver.find_element(by=By.CSS_SELECTOR, value=".recharts-curve.recharts-area-curve")
d_element = time_text.get_attribute("d")
modified_string = re.sub(r"[a-zA-Z]", ",", d_element)[1:]
values = modified_string.split(",")[1::6][1:-1]
values = [160 - float(x) for x in values]
max_value = max(values)

#ツイート推移のデータを計算
values = [round((x / max_value) * tweet_count) for x in values]

In [ ]:
#時間インデックスの取得
now = datetime.now()
start_time = now - timedelta(days = 7, hours=-2, minutes=now.minute, seconds=now.second, microseconds=now.microsecond)
hourly_list = [start_time + timedelta(hours=i) for i in range(24*7-1)]

In [ ]:
#結果の表示
s = pd.Series(values, index=hourly_list)
s.plot()

#WebDriverを終了
#driver.quit()